# Modelisation

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
from sklearn import set_config

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 5000)
pd.set_option("display.max_rows", 101)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

set_config(transform_output="pandas")

In [ ]:
from utils import RANDOM_STATE, TARGET


X = pd.read_csv("./data/kaggle_train_set.csv")

# Supprimons les outliers
large_property_index = X[(X.GrLivArea > 4500) | (X.LotArea >150_000)].index.to_list()
X.drop(index=large_property_index, inplace=True)

y = X.pop(TARGET)

print(f"\nLa shape de X est {X.shape}")
print(f"La shape de y est {y.shape}")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, RobustScaler, StandardScaler, OrdinalEncoder

neighboors_categories = ['MeadowV', 'IDOTRR', 'BrDale', 'Edwards', 'BrkSide', 'OldTown', 'Blueste', 'Sawyer', 'SWISU', 'NAmes', 'NPkVill', 'Mitchel', 'SawyerW', 'Gilbert', 'Blmngtn', 'NWAmes', 'CollgCr', 'ClearCr', 'Crawfor', 'Somerst', 'Timber', 'Veenker', 'StoneBr', 'NoRidge', 'NridgHt']
quality_order = ['Fa', 'TA', 'Gd', 'Ex']

numerical_pipeline = Pipeline(
    steps=[
        ('power_transformation', PowerTransformer(method='yeo-johnson')),
    ]
    
)

categorical_encoding_pipeline = Pipeline(
    steps=[
        ('encoder', OrdinalEncoder(categories=[neighboors_categories, quality_order, quality_order])),
    ]
    
)


preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', categorical_encoding_pipeline, ['Neighborhood', 'ExterQual', 'KitchenQual']),
        ('numerical', numerical_pipeline, ['GarageCars', 'GrLivArea', '1stFlrSF']),
        ('passthrough_selected', 'passthrough', ['OverallQual', 'YearBuilt'])
    ],
    remainder='drop',
    verbose_feature_names_out=False
)


preprocessing

In [ ]:
preprocessing.fit_transform(X)

In [ ]:
from lightgbm import LGBMRegressor
import numpy as np
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

models = {
    "Ridge": Ridge(random_state=RANDOM_STATE),
    "Random Forest": RandomForestRegressor(random_state=RANDOM_STATE),
    "Gradient Boosting": GradientBoostingRegressor(loss='huber', random_state=RANDOM_STATE),
    "LGBM": LGBMRegressor(verbose=0, random_state=RANDOM_STATE),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor()
}

results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessing),
        ('regressor', TransformedTargetRegressor(
            regressor=model,
            func=np.log1p, inverse_func=np.expm1
        ))
    ])
    
    # Validation croisée pour calculer la MAE
    scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')
    results[name] = -scores.mean()


print("Résultats des modèles :")
for name, score in results.items():
    print(f"{name}: MAE = {score:.2f}")

Gradient Boosting performe mieux que les autres. Gardons le pour la suite.

In [ ]:
import optuna


def objective(trial):
    # Définir les hyperparamètres à optimiser
    gb_params = dict(
        max_depth=trial.suggest_int("max_depth", 2, 10),
        loss=trial.suggest_categorical("loss", ['squared_error', 'absolute_error', 'huber', 'quantile']),
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        n_estimators=trial.suggest_int("n_estimators", 100, 1000),
        subsample=trial.suggest_float("subsample", 0.5, 1.0),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 20),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 20),
        max_features=trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    )
    
    # Créer le pipeline avec le modèle et le prétraitement
    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessing),
        ('regressor', TransformedTargetRegressor(
            regressor=GradientBoostingRegressor(random_state=RANDOM_STATE, **gb_params),
            func=np.log1p, inverse_func=np.expm1
        ))
    ])
    
    # Validation croisée pour calculer la MAE
    scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')
    return -scores.mean()

# Lancer l'optimisation avec Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

# Afficher les meilleurs paramètres
best_params = study.best_params
print("Meilleurs hyperparamètres :", best_params)
print("Meilleurs scores: ", study.best_value)

In [ ]:
from sklearn import clone
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

# paramètres
min_samples_for_cv = 5  # si < 5 samples => pas de CV locale, on utilise la prédiction globale
top_k = 10              # afficher les top k gaps positifs

# s'assurer que l'estimator global est entraîné
estimator = Pipeline(
    steps=[
        ('preprocessing', preprocessing),
        ('regressor', TransformedTargetRegressor(
            regressor=GradientBoostingRegressor(random_state=RANDOM_STATE, **best_params),
            func=np.log1p, inverse_func=np.expm1
        ))
    ],
)
estimator.fit(X, y)

# prédictions globales (fallback)
global_preds = pd.Series(estimator.predict(X), index=X.index)

rows = []
for neigh, idx in X.groupby('Neighborhood').groups.items():
    Xn, yn = X.loc[idx], y.loc[idx]
    n = len(Xn)
    if n == 0:
        continue

    # si pas assez de samples ou y constant => on n'effectue pas la CV locale
    if n < min_samples_for_cv or yn.nunique() == 1:
        mae = mean_absolute_error(yn, global_preds.loc[idx])
        method = 'global_pred_no_cv' if n < min_samples_for_cv else 'global_pred_constant_y'
    else:
        cv = min(5, n)
        try:
            scores = cross_val_score(clone(estimator), Xn, yn, cv=cv, scoring='neg_mean_absolute_error')
            mae = -scores.mean()
            method = 'local_cv'
        except Exception:
            mae = mean_absolute_error(yn, global_preds.loc[idx])
            method = 'fallback_global_pred'

    rows.append({
        'Neighborhood': neigh,
        'n_samples': n,
        'mae': mae,
        'gap_vs_global_best': mae - study.best_value,
        'method': method
    })

neigh_local_mae_df = pd.DataFrame(rows).sort_values('gap_vs_global_best', ascending=False).reset_index(drop=True)
neigh_local_mae_df.set_index('Neighborhood', inplace=True)

# quartiers avec gap positif triés par gap décroissant
positive_gaps = neigh_local_mae_df[neigh_local_mae_df['gap_vs_global_best'] > 0].sort_values('gap_vs_global_best', ascending=False)

In [ ]:
print(f"Neighborhoods avec gap positif (pire que study.best_value) :")
display(positive_gaps)

In [ ]:
# Entrainement sur l´ensemble du dataset
from sklearn.metrics import make_scorer
from sklearn.model_selection import TunedThresholdClassifierCV

best_params = {'max_depth': 5, 'loss': 'huber', 'learning_rate': 0.015996481097870868, 'n_estimators': 385, 'subsample': 0.7656864272872252, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'log2'}

estimator = Pipeline(
    steps=[
        ('preprocessing', preprocessing),
        ('regressor', TransformedTargetRegressor(
            regressor=GradientBoostingRegressor(random_state=RANDOM_STATE, **best_params),
            func=np.log1p, inverse_func=np.expm1))
        ]
)

estimator.fit(X, y)

# Load the data
test = pd.read_csv("./data/kaggle_test_set.csv")

# Prédictions
result = estimator.predict(test)

# Créer le DataFrame de soumission avec "ID" comme index
submission_df = pd.DataFrame(result, columns=["SalePrice"], index=test.index)
submission_df.index.name = "ID"  # Renommer l'index en "ID"

# Sauvegarder le fichier CSV
submission_df.to_csv("./data/submission.csv")

# Vérification
check_df = pd.read_csv("./data/submission.csv", index_col='ID')
check_df.head()

In [ ]:
from datetime import datetime
import io
import subprocess
import time

from utils import sauvegarder_model

competition = "regression-prediction-prix-immobilier"


def all_status_complete(df):
    return all(df['status'].isin(['SubmissionStatus.ERROR', 'SubmissionStatus.COMPLETE']))

# Fonction pour relancer le subprocess et récupérer les données
def relaunch_subprocess():
    result = subprocess.run(
        ["kaggle", "competitions", "submissions", "-v", "-c",  competition],
        capture_output=True,
        text=True
        )
    return result.stdout



now = str(datetime.now())
# Timeout de 1 minute
timeout = 60  # en secondes
start_time = time.time()

data = None

if SUBMIT:= True:
    file_path = "./data/submission.csv"
    message = f"timestamp: {now}, Utilisation des paramètres: {estimator.get_params()}"

    result = subprocess.run(
        ["kaggle", "competitions", "submit", "-c", competition, "-f", file_path, "-m", message],
        capture_output=True,
        text=True
    )

    print("⚒️", result.stdout)

    if result.stderr != '':
        print(result.stderr)
    data = None

    while True:
        # Récupérer les données depuis le subprocess
        stdout_data = relaunch_subprocess()
        
        # Convertir les données en DataFrame
        data = pd.read_csv(io.StringIO(stdout_data), parse_dates=['date'])
        
        # Vérifier si tous les statuts sont complets
        if all_status_complete(data):
            break
        
        # Vérification du timeout
        if time.time() - start_time > timeout:
            print("")
            raise RuntimeError("imeout atteint. Arrêt de la boucle.")
        
        # Pause avant la prochaine vérification
        time.sleep(20)  # Pause de 20 secondes

    if data is not None and all_status_complete(data):
        # Trouver la ligne avec la date la plus récente
        most_recent = data.sort_values('date', ascending=False).iloc[0]
        recent_score = most_recent['publicScore']
        best_score = data['publicScore'].min()

        if recent_score < best_score:
            print(f"🥳 Nouveau meilleur score : {recent_score:.5f}")
            sauvegarder_model(estimator, timestamp=now, only_latest=False)
        else:
            print(f"❌ Bien essayé mais c´est moins bon. Score: {recent_score:.5f}. Meilleur score : {best_score:.5f}")

if not SUBMIT:
    sauvegarder_model(estimator, timestamp=now, only_latest=True)
